In [1]:
import os
import numpy as np 
import pandas as pd
from glob import glob
import cv2
import random
import matplotlib.pyplot as plt
import joblib
from tensorflow.keras.models import load_model

In [2]:
dtype_dict = {
    'code': str
}

In [3]:
test_list = pd.read_csv(r'C:\Users\USER\Paper\with index_V3\test_path.csv', dtype=dtype_dict).iloc[:,1:]

test_path = r'C:\Users\USER\Paper\with index_V3\Test\\'
test_list['File Paths'] = test_list['File Paths'].apply(lambda path:os.path.join(test_path, path))
df_drop_test = test_list.drop_duplicates('code',keep='first').reset_index(drop=True)

In [4]:
len(df_drop_test)

1778

In [5]:
model = load_model(r'C:\Users\USER\Paper\with index_V3\model_V2.h5')

In [6]:
backtest = pd.DataFrame()
lower = 1700
upper = lower+100
for code in df_drop_test['code'].values[lower:upper]:
    test_df = test_list[test_list['code'] == code].reset_index(drop=True)

    img_list_test = []
    for i in range(len(test_df)):
        img_test = cv2.imread(test_df['File Paths'].loc[i])

        img_list_test.append(img_test)
    X_test = np.concatenate(img_list_test, axis = 0)

    X_test = X_test.reshape(len(test_df), 170,340,3)/255
    y_prob = model.predict(X_test)
    y_prob_flat = np.ravel(y_prob)
    y_prob_df = pd.DataFrame({'predict_rate':y_prob_flat})
    y_prob_df['Decision'] = y_prob_df['predict_rate'].apply(lambda value:'Sell' if value<=0.001 else ('Buy' if value >= 0.995 else 'No Action'))

    backtest_df = pd.concat([test_df['code'],
                             test_df['Date'],
                             y_prob_df['Decision'],
                             y_prob_df['predict_rate'],
                             test_df['Open'],
                             test_df['High'],
                             test_df['Low'],
                             test_df['Close'],],axis=1)
    backtest_df['RR'] = 0
    if not backtest_df[backtest_df['Decision'] == 'Buy'].empty and not backtest_df[backtest_df['Decision'] == 'Sell'].empty:
        buy_index = backtest_df[backtest_df['Decision'] == 'Buy'].index[0]
        sell_index = backtest_df[backtest_df['Decision'] == 'Sell'].index[0]

        if buy_index > sell_index:
            backtest_df.loc[sell_index, 'RR'] = float(backtest_df.loc[sell_index, 'Close'] /backtest_df.loc[buy_index, 'Close'] - 1)
        elif buy_index < sell_index:
            backtest_df.loc[buy_index, 'RR']  = float(backtest_df.loc[sell_index, 'Close']/backtest_df.loc[buy_index, 'Close'] - 1)
    backtest = pd.concat([backtest, backtest_df], ignore_index=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_8520\3459300542.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.24755685595286' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  backtest_df.loc[buy_index, 'RR']  = float(backtest_df.loc[sell_index, 'Close']/backtest_df.loc[buy_index, 'Close'] - 1)
C:\Users\USER\AppData\Local\Temp\ipykernel_8520\3459300542.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.010776620370370416' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  backtest_df.loc[sell_index, 'RR'] = float(backtest_df.loc[sell_index, 'Close'] /backtest_df.loc[buy_index, 'Close'] - 1)
C:\Users\USER\AppData\Local\Temp\ipykernel_8520\3459300542.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pan

In [7]:
backtest.to_csv('backtest_' + str(lower) +'_' + str(upper) + '.csv')